In [ ]:
'''
all the model tuned
balanced data used (BalancedMRIDataset)
'''

In [1]:
import os

import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from config import Config, Device
from datasets import MRIDataset, BalancedMRIDataset
from models import MriResentModel
from trainer import Trainer
from tester import Tester

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
device = Device.device
print(device)

mps


In [45]:
data_path = os.path.join(os.getcwd(), "data")
labels_path = "train.csv"

batch_size = Config.batch_size
num_epochs = Config.num_epochs
learning_rate = Config.learning_rate
mean = Config.mean # mean of the entire datasaet
std = Config.std # std of the entire dataaset
image_size = 224

In [46]:

resclaed_mean = round(mean/255,4) # re-scale the actual mean
rescaled_std = round(std/255, 4) # re-scale the actual std

train_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

augment_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

test_transforms = transforms.Compose([
    # transforms.Lambda(lambda img: img.astype(np.float32)),
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

In [47]:
train_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='train',
    transform=train_transforms,
    augment_transform=augment_transforms,
    augment=True,
    max_slices=20
)

val_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='val',
    transform=test_transforms,
    max_slices=20
)

test_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='test',
    transform=test_transforms,
    max_slices=20
)

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=32)
test_dl = DataLoader(test_dataset, batch_size=32)

In [48]:
data_, label_ = next(iter(train_dl))
data_.size()

torch.Size([32, 20, 224, 224])

In [ ]:
model = MriResentModel(1,1).to(device)
model

In [8]:
def compute_class_weights_from_csv(csv_file_path):
    # Read the CSV file
    df = pd.read_csv(csv_file_path)

    labels = df['prediction'].values

    # Convert labels to integers if they are not already
    labels = labels.astype(int)

    # Compute class weights
    unique_labels = np.unique(labels)
    class_weights = compute_class_weight(
        class_weight='balanced', classes=unique_labels, y=labels)

    # Convert to torch tensor
    return torch.tensor(class_weights, dtype=torch.float)


# Path to your CSV file
class_weights = compute_class_weights_from_csv(labels_path)
print(class_weights)
# For binary classification, use the appropriate class weight
# Assuming binary classification with class labels 0 and 1
class_weights = class_weights[1]  # Adjust if necessary
print("Class Weights:", class_weights)

tensor([0.5713, 4.0051])
Class Weights: tensor(4.0051)


In [41]:
# loss and optimizer

# criterion = nn.BCEWithLogitsLoss().to(device)
# criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [50]:
model_name = model.__class__.__name__
model_name

'MriResentModel'

In [51]:
trainer = Trainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_dl=train_dl,
    val_dl=val_dl,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    num_epochs=4,
    patience=5,
    threshold=0.5,
    save_path=f"saved_models/{model_name}.pth"
)

# Start training
trainer.train()

100%|██████████| 89/89 [17:57<00:00, 12.11s/it]


Confusion Matrix:
[[10960     0]
 [ 1560     0]]
Epoch 1/4, Train Loss: 1222.8494, Train Accuracy: 0.5606
Epoch 1/4, Val Accuracy: 17.5080, Precision: 0.0000, Recall: 0.0000, AUC: 0.5000, Avg Metric: 5.8360


100%|██████████| 89/89 [17:51<00:00, 12.04s/it]


Confusion Matrix:
[[10960     0]
 [ 1560     0]]
Epoch 2/4, Train Loss: 1219.0895, Train Accuracy: 0.5677
Epoch 2/4, Val Accuracy: 17.5080, Precision: 0.0000, Recall: 0.0000, AUC: 0.5000, Avg Metric: 5.8360


100%|██████████| 89/89 [18:09<00:00, 12.24s/it]


Confusion Matrix:
[[10960     0]
 [ 1560     0]]
Epoch 3/4, Train Loss: 1222.0585, Train Accuracy: 0.5574
Epoch 3/4, Val Accuracy: 17.5080, Precision: 0.0000, Recall: 0.0000, AUC: 0.5000, Avg Metric: 5.8360


100%|██████████| 89/89 [18:04<00:00, 12.19s/it]


Confusion Matrix:
[[10960     0]
 [ 1560     0]]
Epoch 4/4, Train Loss: 1220.9354, Train Accuracy: 0.5500
Epoch 4/4, Val Accuracy: 17.5080, Precision: 0.0000, Recall: 0.0000, AUC: 0.5000, Avg Metric: 5.8360


In [12]:
model = MriResentModel(1,1).to(device)
model.load_state_dict(torch.load(f"saved_models/{model_name}.pth"))

/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fateme/Projects/iaaa-project/iaaa-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [13]:
tester = Tester(
    model=model,
    criterion = criterion,
    test_dl=test_dl,
    test_dataset=test_dataset,
    device=device,
    threshold=0  # Set the threshold for binary classification
)

# Perform testing and print metrics
tester.test()

Test Accuracy: 0.7604, Precision: 0.2049, Recall: 0.3205, AUC: 0.5926, Avg Metric: 0.3727
